# Compare results of solvers 

## import library

In [3]:
import os
import re
import subprocess
import time
import datetime as dt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from itertools import permutations
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

##　パスの指定

In [30]:
data_dir_path = "../../tests/kouei_data/検証用データ1/0530データ-20240610-135109304-TES1EDI001-TES1AP001"
save_dir_path = "../../tests/kouei_data_test"
instance_name = "0530"

In [5]:
fig = px.scatter(
    input_df, 
    x="LATITUDE", y="LONGITUDE", 
    text=input_df['CUST NO.'], 
    width=1000, height=800)
fig.show()

NameError: name 'input_df' is not defined

In [ ]:
# save
img_file_path = os.path.join(
    DATA_DIR_PATH, f'{INSTANCE_NAME}.png'
)
fig.write_image(img_file_path)

NameError: name 'DATA_DIR_PATH' is not defined

##  ファイル内検索 + ファイルパス取得

## クラス化

In [26]:
class FileFinder:
    def __init__(self, directory, file_name):
        self.directory = directory
        self.file_name = file_name

    def find_csv_file_and_return_path(self):
        for root, dirs, files in os.walk(self.directory):
            for file in files:
                if file.endswith(f"{self.file_name}.csv"):
                    current_directory = os.getcwd()  # 現在のディレクトリを取得
                    relative_path = os.path.relpath(os.path.join(root, file), current_directory)  # 指定したディレクトリから現在のディレクトリまでの相対パスを取得
                    return relative_path  # 相対パスを返す
                    # TODO:保存先のディレクトリ指定

### クラステスト

In [27]:
file_name = "input"

finder = FileFinder(data_dir_path, file_name)
result = finder.find_csv_file_and_return_path()

input_df = pd.read_csv(
    result)
input_df['FROM TIME'] = '04:00'
input_df['TO TIME'] = '16:00'
input_df

,CUST NO.,LATITUDE,LONGITUDE,DEMAND,FROM TIME,TO TIME,SERVICE TIME
0,0,35.691263,139.677831,0,04:00,16:00,0
1,1,35.721917,139.912438,0,04:00,16:00,20
2,2,35.684535,139.916100,0,04:00,16:00,20
3,3,35.683649,139.987878,0,04:00,16:00,20
4,4,35.637891,140.060469,0,04:00,16:00,20
...,...,...,...,...,...,...,...
144,144,35.674983,140.045395,0,04:00,16:00,20
145,145,35.717491,140.007313,0,04:00,16:00,20
146,146,35.838214,139.967206,0,04:00,16:00,20
147,147,35.695474,140.543591,0,04:00,16:00,20


In [9]:
request_data_path = "../../tests/kouei_data/検証用データ1/20240530元ネタ.csv"
request_df = pd.read_csv(
    request_data_path, encoding='shift-jis')
request_df

,センターコード,オーダーフラグ,確定フラグ,配送希望日,届先ID,届先名1,届先名2,届先郵便番号,届先住所1,届先住所2,...,明細容量,明細荷姿数,商品区分,出発デポコード,到着デポコード,受注日,受注時刻,処理コード,荷主コード,集荷配送区分
0,888,1,1,2024/5/30,763733,市川市新田３,NaN,NaN,千葉県市川市新田３丁目１３番地７号,NaN,...,0.36,6,NaN,1,1,2024/5/29,16:00,A,NaN,2
1,888,1,1,2024/5/30,763733,市川市新田３,NaN,NaN,千葉県市川市新田３丁目１３番地７号,NaN,...,0.36,6,NaN,1,1,2024/5/29,16:00,A,NaN,2
2,888,1,1,2024/5/30,763733,市川市新田３,NaN,NaN,千葉県市川市新田３丁目１３番地７号,NaN,...,0.36,6,NaN,1,1,2024/5/29,16:00,A,NaN,2
3,888,1,1,2024/5/30,763733,市川市新田３,NaN,NaN,千葉県市川市新田３丁目１３番地７号,NaN,...,0.36,6,NaN,1,1,2024/5/29,16:00,A,NaN,2
4,888,1,1,2024/5/30,763733,市川市新田３,NaN,NaN,千葉県市川市新田３丁目１３番地７号,NaN,...,0.36,6,NaN,1,1,2024/5/29,16:00,A,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,888,1,1,2024/5/30,997604,千葉市花見川区幕張本郷６,NaN,NaN,千葉県千葉市花見川区幕張本郷６丁目２４番地２１号,NaN,...,1.80,30,NaN,1,1,2024/5/29,16:00,A,NaN,2
660,888,1,1,2024/5/30,997624,船橋市西船橋４２７２１,NaN,NaN,千葉県船橋市西船橋４丁目２７番地２号西船橋駅前プラザビル１階,NaN,...,1.80,30,NaN,1,1,2024/5/29,16:00,A,NaN,2
661,888,1,1,2024/5/30,997634,柏市新柏１１５２４,NaN,NaN,千葉県柏市新柏１丁目1524-3,NaN,...,1.80,30,NaN,1,1,2024/5/29,16:00,A,NaN,2
662,888,1,1,2024/5/30,998744,匝瑳市八日市場イ７０９,NaN,NaN,千葉県匝瑳市八日市場イ７０９番地１号,NaN,...,1.44,24,NaN,1,1,2024/5/29,16:00,A,NaN,2


In [23]:
# 重複を削除して届先IDごとのユニークな値をリストとして取得
unique_dest_df = request_df.drop_duplicates(subset=['届先ID'])['オーダー重量']
# 0行目に挿入
new_row = pd.Series(0)
unique_dest_df = pd.concat([new_row, unique_dest_df]).reset_index(drop=True)
unique_dest_df

0        0.00
1      452.28
2       60.00
3      720.00
4      180.00
        ...  
144    300.00
145    300.00
146    300.00
147    240.00
148    120.00
Length: 149, dtype: float64

In [28]:
input_df['DEMAND'] = unique_dest_df
input_df

,CUST NO.,LATITUDE,LONGITUDE,DEMAND,FROM TIME,TO TIME,SERVICE TIME
0,0,35.691263,139.677831,0.00,04:00,16:00,0
1,1,35.721917,139.912438,452.28,04:00,16:00,20
2,2,35.684535,139.916100,60.00,04:00,16:00,20
3,3,35.683649,139.987878,720.00,04:00,16:00,20
4,4,35.637891,140.060469,180.00,04:00,16:00,20
...,...,...,...,...,...,...,...
144,144,35.674983,140.045395,300.00,04:00,16:00,20
145,145,35.717491,140.007313,300.00,04:00,16:00,20
146,146,35.838214,139.967206,300.00,04:00,16:00,20
147,147,35.695474,140.543591,240.00,04:00,16:00,20


In [31]:
# save
input_file_path = os.path.join(
    save_dir_path, f'request_cns_solver_{instance_name}_input.csv'
)
input_df.to_csv(input_file_path, index=False)

In [ ]:
directory = "../../tests/kouei_data/検証用データ1/0530データ-20240610-135109304-TES1EDI001-TES1AP001"
input_file_path = FileFinder(directory, "input").find_csv_file_and_return_path()
distance_file_path = FileFinder(directory, "dist").find_csv_file_and_return_path()
time_file_path = FileFinder(directory, "time").find_csv_file_and_return_path()
vs_file_path = FileFinder(directory, "vs").find_csv_file_and_return_path()
output_file_path = directory

## ヘテロ車両定義ファイル・ファイルパス作成

In [ ]:
headers = ['LOAD_LIMIT', 'E-CAPACITY', 'E-INITIAL', 'E-COST', 'E-MARGIN', 'VTYPE', 'ETYPE']
vehicle_setting_df = pd.DataFrame(columns=headers)
# TODO: 重量の順番を考慮して計算
# LOAD_LIMITが13tである5行のデータを作成
vehicle_data_13t = {
    # 'VEHICLENO.': list(range(0, 5)),
    'LOAD_LIMIT': [12800] * len(range(0, 5)),
    'E-CAPACITY': [0] * len(range(0, 5)),
    'E-INITIAL': [0] * len(range(0, 5)),
    'E-COST': [0] * len(range(0, 5)),
    'E-MARGIN': [0] * len(range(0, 5)),
    'VTYPE': [''] * len(range(0, 5)),
    'ETYPE': [''] * len(range(0, 5)),
}

# LOAD_LIMITが8tである10行のデータを作成
vehicle_data_8t = {
    # 'VEHICLENO.': list(range(0, 10)),
    'LOAD_LIMIT': [7800] * 10,
    'E-CAPACITY': [0] * 10,
    'E-INITIAL': [0] * 10,
    'E-COST': [0] * 10,
    'E-MARGIN': [0] * 10,
    'VTYPE': [''] * 10,
    'ETYPE': [''] * 10,
    'LOAD_LIMIT2': [0] * 10
}

# LOAD_LIMITが4tである10行のデータを作成
vehicle_data_4t = {
    # 'VEHICLENO.': list(range(0, 10)),
    'LOAD_LIMIT': [3800] * 10,
    'E-CAPACITY': [0] * 10,
    'E-INITIAL': [0] * 10,
    'E-COST': [0] * 10,
    'E-MARGIN': [0] * 10,
    'VTYPE': [''] * 10,
    'ETYPE': [''] * 10,
    'LOAD_LIMIT2': [0] * 10
}

# LOAD_LIMITが3tである15行のデータを作成
vehicle_data_3t = {
    # 'VEHICLENO.': list(range(0, 15)),
    'LOAD_LIMIT': [2800] * 15,
    'E-CAPACITY': [0] * 15,
    'E-INITIAL': [0] * 15,
    'E-COST': [0] * 15,
    'E-MARGIN': [0] * 15,
    'VTYPE': [''] * 15,
    'ETYPE': [''] * 15,
    'LOAD_LIMIT2': [0] * 15
}
# 4つのデータセットをDataFrameに変換し、結合
vehicle_df_13t = pd.DataFrame(vehicle_data_13t, columns=headers)
vehicle_df_8t = pd.DataFrame(vehicle_data_8t, columns=headers)
vehicle_df_4t = pd.DataFrame(vehicle_data_4t, columns=headers)
vehicle_df_3t = pd.DataFrame(vehicle_data_3t, columns=headers)

new_rows = pd.concat([vehicle_df_13t, vehicle_df_8t, vehicle_df_4t, vehicle_df_3t], ignore_index=True)
vehicle_setting_df = pd.concat([vehicle_setting_df, new_rows], ignore_index=True)
vehicle_setting_df['VEHICLENO.'] = vehicle_setting_df.index
vehicle_setting_df = vehicle_setting_df.reindex(columns=['VEHICLENO.'] + headers)
vehicle_setting_df
# vehicle_setting_df.to_csv(vs_file_path, index=False)

,VEHICLENO.,LOAD_LIMIT,E-CAPACITY,E-INITIAL,E-COST,E-MARGIN,VTYPE,ETYPE
0,0,12800,0,0,0,0,,
1,1,12800,0,0,0,0,,
2,2,12800,0,0,0,0,,
3,3,12800,0,0,0,0,,
4,4,12800,0,0,0,0,,
5,5,7800,0,0,0,0,,
6,6,7800,0,0,0,0,,
7,7,7800,0,0,0,0,,
8,8,7800,0,0,0,0,,
9,9,7800,0,0,0,0,,


## クラス化

In [34]:
class VehicleDataProcessor:
    def __init__(self):
        self.headers = ['LOAD_LIMIT', 'E-CAPACITY', 'E-INITIAL', 'E-COST', 'E-MARGIN', 'VTYPE', 'ETYPE']
        self.vehicle_setting_df = pd.DataFrame(columns=self.headers)

    def create_vehicle_data(self, load_limit, count, vehicle_type=''):
        vehicle_data = {
            'LOAD_LIMIT': [load_limit] * count,
            'E-CAPACITY': [0] * count,
            'E-INITIAL': [0] * count,
            'E-COST': [0] * count,
            'E-MARGIN': [0] * count,
            'VTYPE': [vehicle_type] * count,
            'ETYPE': [''] * count,
        }
        return pd.DataFrame(vehicle_data, columns=self.headers)

    def process_vehicle_data(self, *args):
        all_dfs = [self.vehicle_setting_df] + list(args)
        self.vehicle_setting_df = pd.concat(all_dfs, ignore_index=True)
        return self.vehicle_setting_df

# テスト        
processor = VehicleDataProcessor()
vehicle_df_13t = processor.create_vehicle_data(12800, 5)
vehicle_df_8t = processor.create_vehicle_data(7800, 10)
vehicle_df_4t = processor.create_vehicle_data(3800, 10)
vehicle_df_3t = processor.create_vehicle_data(2800, 15)
processed_vehicle_data = processor.process_vehicle_data(vehicle_df_13t, vehicle_df_8t, vehicle_df_4t, vehicle_df_3t)
processed_vehicle_data


,LOAD_LIMIT,E-CAPACITY,E-INITIAL,E-COST,E-MARGIN,VTYPE,ETYPE
0,12800,0,0,0,0,,
1,12800,0,0,0,0,,
2,12800,0,0,0,0,,
3,12800,0,0,0,0,,
4,12800,0,0,0,0,,
5,7800,0,0,0,0,,
6,7800,0,0,0,0,,
7,7800,0,0,0,0,,
8,7800,0,0,0,0,,
9,7800,0,0,0,0,,


In [35]:
# save
vs_file_path = os.path.join(
    save_dir_path, f'request_cns_solver_{instance_name}_vs.csv'
)
processed_vehicle_data.to_csv(vs_file_path, index=False)

# CNS Solver

In [ ]:
std_out = subprocess.check_output([
        "pypy","../../CNSsolver_20230602_v2303/vrp_classical/code/main_tw.py",
        "-i", input_file_path,
        "-d", distance_file_path,
        "-t", time_file_path,
        "-o", output_file_path,
        "-v",  40, 
        "-vs", vs_file_path,
        "-rv", # reduce vehicle
        "-s", 6:00,
        "-e", 16:30,
        "-mt", #multi-trip (rotation)
        # "-dem", str(capacity),
        "-lpt", "hard", # hard constraint on package weight v.s. vehicle capacity
        #"-avedistper", "10"
        # "-l" #1.20.2022
        # "-multithread", "0"
        "ls",
        "-l",
        ]  # , text=True
    )
# 標準出力をbytes型で受け取るには下記のコードを追加する
# (['ls', '-l'])
# 上記にtext=Trueを付けるとstr型になる

# save std_out as txt file
with open(
        f"results/simulation/{cust_name}/{instance_name}_log_CARG.txt", "wb"
    ) as file:
        file.write(std_out)